In [11]:
from deprecat.sphinx import deprecat
@deprecat(
    reason="""i dont like n, im so so so so so so so so tired
more lines
more line
liney
""", deprecated_arg="alpha",
    version='0.3.0')
@deprecat(
    reason="""i dont like n, im so so so so so so so so tired
more lines
more line
liney
""", deprecated_arg="n",
    version='0.3.0')
class BetaBinomial:
    R"""
    Beta-binomial log-likelihood.

    Equivalent to binomial random variable with success probability
    drawn from a beta distribution.
    The pmf of this distribution is

    .. math::

       f(x \mid \alpha, \beta, n) =
           \binom{n}{x}
           \frac{B(x + \alpha, n - x + \beta)}{B(\alpha, \beta)}

    .. plot::

        import matplotlib.pyplot as plt
        import numpy as np
        import scipy.stats as st
        from scipy import special
        import arviz as az
        plt.style.use('arviz-darkgrid')

        def BetaBinom(a, b, n, x):
            pmf = special.binom(n, x) * (special.beta(x+a, n-x+b) / special.beta(a, b))
            return pmf

        x = np.arange(0, 11)
        alphas = [0.5, 1, 2.3]
        betas = [0.5, 1, 2]
        n = 10
        for a, b in zip(alphas, betas):
            pmf = BetaBinom(a, b, n, x)
            plt.plot(x, pmf, '-o', label=r'$\alpha$ = {}, $\beta$ = {}, n = {}'.format(a, b, n))
        plt.xlabel('x', fontsize=12)
        plt.ylabel('f(x)', fontsize=12)
        plt.ylim(0)
        plt.legend(loc=9)
        plt.show()

    ========  =================================================================
    Support   :math:`x \in \{0, 1, \ldots, n\}`
    Mean      :math:`n \dfrac{\alpha}{\alpha + \beta}`
    Variance  :math:`n \dfrac{\alpha \beta}{(\alpha+\beta)^2 (\alpha+\beta+1)}`
    ========  =================================================================

    Parameters
    ----------
    n: int
        Number of Bernoulli trials (n >= 0).
    alpha: float
        alpha > 0.
    beta: float
        beta > 0.
    """

    @classmethod
    def dist(cls, alpha, beta, n, *args, **kwargs):
        alpha = at.as_tensor_variable(floatX(alpha))
        beta = at.as_tensor_variable(floatX(beta))
        n = at.as_tensor_variable(intX(n))
        return super().dist([n, alpha, beta], **kwargs)

    def logp(value, n, alpha, beta):
        r"""
        
        Calculate log-probability of BetaBinomial distribution at specified value.

        Parameters 
        ----------
        value: numeric
            Value(s) for which log-probability is calculated. If the log probabilities for multiple
            values are desired the values must be provided in a numpy array or Aesara tensor
        
            .. warning::
               value is useless remove it

        anything: str
            this is something

        Returns
        -------
        TensorVariable
        
        """
        return bound(
            binomln(n, value) + betaln(value + alpha, n - value + beta) - betaln(alpha, beta),
            value >= 0,
            value <= n,
            alpha > 0,
            beta > 0,
        )

    def logcdf(value, n, alpha, beta):
        """
        Compute the log of the cumulative distribution function for BetaBinomial distribution
        at the specified value.

        Parameters
        ----------
        value: numeric
            Value for which log CDF is calculated.

        Returns
        -------
        TensorVariable
        """
        # logcdf can only handle scalar values at the moment
        if np.ndim(value):
            raise TypeError(
                f"BetaBinomial.logcdf expects a scalar value but received a {np.ndim(value)}-dimensional object."
            )

        safe_lower = at.switch(at.lt(value, 0), value, 0)

        return bound(
            at.switch(
                at.lt(value, n),
                at.logsumexp(
                    BetaBinomial.logp(at.arange(safe_lower, value + 1), n, alpha, beta),
                    keepdims=False,
                ),
                0,
            ),
            0 <= value,
            0 <= n,
            0 < alpha,
            0 < beta,
        )


In [12]:
print(BetaBinomial.__doc__)


Beta-binomial log-likelihood.

Equivalent to binomial random variable with success probability
drawn from a beta distribution.
The pmf of this distribution is

.. math::

   f(x \mid \alpha, \beta, n) =
       \binom{n}{x}
       \frac{B(x + \alpha, n - x + \beta)}{B(\alpha, \beta)}

.. plot::

    import matplotlib.pyplot as plt
    import numpy as np
    import scipy.stats as st
    from scipy import special
    import arviz as az
    plt.style.use('arviz-darkgrid')

    def BetaBinom(a, b, n, x):
        pmf = special.binom(n, x) * (special.beta(x+a, n-x+b) / special.beta(a, b))
        return pmf

    x = np.arange(0, 11)
    alphas = [0.5, 1, 2.3]
    betas = [0.5, 1, 2]
    n = 10
    for a, b in zip(alphas, betas):
        pmf = BetaBinom(a, b, n, x)
        plt.plot(x, pmf, '-o', label=r'$\alpha$ = {}, $\beta$ = {}, n = {}'.format(a, b, n))
    plt.xlabel('x', fontsize=12)
    plt.ylabel('f(x)', fontsize=12)
    plt.ylim(0)
    plt.legend(loc=9)
    plt.show()

========  =====